# Libraries

In [1]:
import cv2
import numpy as np
import pydub
import sounddevice as sd
from matplotlib import pyplot as plt
from scipy.io import wavfile
import pygame
from tkinter import *

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Image Processing
<div dir='rtl'>
در این بخش تابع های مربوط به پردازش تصویر نت ها را شرح خواهیم داد.
</div>

### Line Detection
<div dir='rtl'>
در این بخش با استفاده از تابع line_detector در پی یافتن مختصات خطوط افقی در صفحه خواهیم بود. این تابع یک خروجی از نوع tuple برمیگرداند که دارای 5 خروجی به شرح زیر می باشد:
<br>
1. یک لیست شامل مختصه عمودی خطوط
<br>
2.فاصله عمودی بین دو خط متوالی
<br>
3. تعداد سری هایی که نت ها در صفحه تکرار شده اند
<br>
4. مختصات افقی انتهایی 5 خط آخر
<br>
5. مختصات افقی بقیه خطوط
</div>

In [2]:
def line_detector(img):
    ls = [] # define a list for saving the vertical positions of lines
    result = img.copy() # make a copy of input image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert input to gray
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40,1))
    detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
    cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # find horizontal lines
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        ls.append(c[0,0,1])
    return (ls, ls[0] - ls[1], len(ls) // 5, cnts[0][1][0][0], cnts[-1][1][0][0])

### Template Matching
<div dir='rtl'>
    در این بخش با استفاده از تابع match_template مختصات نت های مختلف را در تصویر پیدا می کنیم. برای این کار از روشی که در فاز 1 پروژه استفاده شده است استفاده می کنیم اما با این تفاوت که در اینجا پس از محاسبه مختصات نقاط، نقاطی که قبلا مشخص شده اند یا خارج از محدوده ی نت ها هستند را حذف می کنیم.
    <br>
    ورودی های این تابع img به عنوان تصویر ورودی و temp به عنوان تصویری است که دنبال آن می گردیم. پس از این دو مقادیر تعداد سری هایی که نت در صفحه تکرار شده است، مقدار دقت پیدا کردن تصویر، یک لیست که مختصات در آن ذخیره شود، نام نت و مختصات انتهای 5 خط آخر و سایر خطوط می باشد، یک لیست شامل مختصات عمودی خطوط و فاصله عمودی دو خط متوالی آمده است.
</div>

In [3]:
def match_template(img, temp, num, threshold, ls, name, last_length, length, lines, height):
    w, h = temp.shape[::-1] #find height and width of the template
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to gray image
    res = cv2.matchTemplate(gray, temp, cv2.TM_CCOEFF_NORMED) #find the similarity
    loc = np.where(res >= threshold) #find more similar parts
    for pt in zip(*loc[::-1]):
        # remove the repeated parts
        add = True
        for l in ls:
            for it in l:
                if abs(pt[0]-it[0]) <= 5 and abs(pt[1]-it[1]) <= 20:
                    add = False
        if pt[1] + h <= lines[-1] - 3 * height:
            add = False
        if pt[0] <= 140 and pt[1] + h <= lines[(num-1) * 5] + 3*height:
            add = False
        if pt[0] <= 70:
            add = False
        if pt[0] >= last_length - 10 and num > 1 and (lines[0]+3*height-pt[1])//(lines[0]-lines[5]) == 0:
            add = False
        if pt[0] >= last_length - 10 and num == 1:
            add = False
        if pt[0] >= length - 10:
            add = False
        if add:
            # add to the list
            if num > 1:
                ls[(lines[0] + 3*height - pt[1]) // (lines[0] - lines[5])].append((pt[0],pt[1],name))
            else:
                ls[0].append((pt[0],pt[1],name))

### Convert The Points List To Music
<div dir='rtl'>
    در این بخش با استفاده از تابع list_to_music لیست بخش هایی که از تصویر که با نت ها همخوانی داشت و در قسمت قبل پیدا کرده بودیم را به یک لیست music تبدیل می کنیم تا بتوانیم آن را با استفاده از توابع داده شده در فایل پروژه این نت ها را با استفاده از پایتون بنوازیم.
    <br>
    ورودی های تابع لیست قسمت قبل، طول عمودی تصویر های زمینه در قالب یک لیست، تعداد ردیف های نت، لیست music که می خواهیم تشکیل دهیم، مختصات عمودی خطوط و فاصله عمودی خطوط می باشد . همانطور که مشخص است این تابع با گرفتن این ورودی ها و با استفاده از مختصات نسبی نقاط موجود در لیست نسبت به خطوط نوع نت را مشخص می کند.
</div>

In [4]:
def list_to_music(ls, h, num, music, lines, height):
    i = num * 5 #number of lines
    dur_dic = {'C':3, 'M':2, 'S':1} #duration of every note
    height_dic = {'C':h[2], 'M':h[3], 'M1':h[4], 'S':h[5]} #height of templates
    for l in ls:
        i = i - 5
        up = 0 #for (#)s in the image
        down = 0 #for (b)s in the image
        for j in range(len(l)):
            it = l[j] #coordinates of the note
            if up: #if up means if it be a (#) in the last part of loop
                up = 0
                continue
            if down: #if down means if it be a (b) in  the last part of loop
                down = 0
                continue
            if it[2] == 'H': #if it be a (#)
                it = l[j+1]
                up = 1
            if it[2] == 'B': #if it be a (b)
                it = l[j+1]
                down = 1
            dur = dur_dic[it[2][0]] #find duration from the dictionary
            h = height_dic[it[2]] #find height of the template from the dictionary
            #find octaves
            if it[1] + h // 2 <= lines[i] + 3*height and it[1] + h // 2 >= lines[i] + 2*height: #G3
                music.append([10+up,3,dur])
            elif it[1] + h <= lines[i] + 3*height and it[1] + h >= lines[i] + 2*height: #A3
                music.append([up,3,dur])
            elif it[1] + h // 2 <= lines[i] + 2*height and it[1] + h // 2 >= lines[i] + height: #B3
                music.append([2+up,3,dur])
            elif it[1] + h >= lines[i] + height and it[1] + h <= lines[i] + 2*height: #C4
                music.append([3+up,4,dur])
            elif it[1] + h // 2 <= lines[i] + height and it[1] + h // 2 >= lines[i]: #D4
                music.append([5+up,4,dur])
            elif it[1] + h <= lines[i] + height and it[1] + h >= lines[i]: #E4
                music.append([7+up,4,dur])
            elif it[1] + h // 2 <= lines[i] and it[1] + h // 2 >= lines[i] - height: #F4
                music.append([8+up,4,dur])
            elif it[1] + h <= lines[i] and it[1] + h >= lines[i] - height: #G4
                music.append([10+up,4,dur])
            elif it[1] + h // 2 <= lines[i] - height and it[1] + h // 2 >= lines[i] - 2*height: #A4
                music.append([up,4,dur])
            elif it[1] + h <= lines[i] - height and it[1] + h >= lines[i] - 2*height: #B4
                music.append([2+up,4,dur])
            elif it[1] + h // 2 <= lines[i] - 2*height and it[1] + h // 2 >= lines[i] - 3*height: #C5
                music.append([3+up,5,dur])
            elif it[1] + h <= lines[i] - 2*height and it[1] + h >= lines[i] - 3*height: #D5
                music.append([5+up,5,dur])
            elif it[1] + h // 2 <= lines[i] - 3*height and it[1] + h // 2 >= lines[i] - 4*height: #E5
                music.append([7+up,5,dur])
            elif it[1] + h <= lines[i] - 3*height and it[1] + h >= lines[i] - 4*height: #F5
                music.append([8+up,5,dur])
            elif it[1] + h // 2 <= lines[i] - 4*height and it[1] + h // 2 >= lines[i] - 5*height: #G5
                music.append([10+up,5,dur])
            elif it[1] + h <= lines[i] - 4*height and it[1] + h >= lines[i] - 5*height: #A5
                music.append([up,5,dur])
            elif it[1] + h // 2 <= lines[i] - 5*height and it[1] + h // 2 >= lines[i] - 6*height: #B5
                music.append([2+up,5,dur])
            elif it[1] + h <= lines[i] - 5*height and it[1] + h >= lines[i] - 6*height: #C6
                music.append([3+up,6,dur])

### Create The Music
<div dir='rtl'>
    در این بخش با استفاده از تابع final_func و با استفاده از توابع قبلی با ورودی گرفتن تصویر لیست music را تشکیل می دهیم.
</div>

In [5]:
def final_func(img, lines, height, num, last_length, length, ls, music):
    # define templates
    temp0 = cv2.imread("Hash.png",0)
    temp1 = cv2.imread("B.png",0)
    temp2 = cv2.imread('Crotchet.png',0)
    temp3 = cv2.imread('Minim.png',0)
    temp4 = cv2.imread('Minim1.png',0)
    temp5 = cv2.imread('Semibreve.png',0)
    #find heights and widths
    w0, h0 = temp0.shape[::-1]
    w1, h1 = temp1.shape[::-1]
    w2, h2 = temp2.shape[::-1]
    w3, h3 = temp3.shape[::-1]
    w4, h4 = temp4.shape[::-1]
    w5, h5 = temp5.shape[::-1]
    
    #find templates with match_template function
    match_template(img, temp0, num, 0.47, ls, 'H', last_length, length, lines, height)
    match_template(img, temp1, num, 0.8, ls, 'B', last_length, length, lines, height)
    match_template(img, temp2, num, 0.5, ls, 'C', last_length, length, lines, height)
    match_template(img, temp3, num, 0.5, ls, 'M', last_length, length, lines, height)
    match_template(img, temp4, num, 0.8, ls, 'M1', last_length, length, lines, height)
    match_template(img, temp5, num, 0.8, ls, 'S', last_length, length, lines, height)
    
    #sort the list
    for l in ls:
        l.sort()
    #draw rectangle around the templates
    for l in ls:
        for pt in l:
            if pt[2] == 'C':
                cv2.rectangle(img, (pt[0],pt[1]), (pt[0] + w2, pt[1] + h2), (0,0,255), 1)
            elif pt[2] == 'M':
                cv2.rectangle(img, (pt[0],pt[1]), (pt[0] + w3, pt[1] + h3), (0,255,0), 1)
            elif pt[2] == 'M1':
                cv2.rectangle(img, (pt[0],pt[1]), (pt[0] + w4, pt[1] + h4), (0,255,0), 1)
            elif pt[2] == 'S':
                cv2.rectangle(img, (pt[0],pt[1]), (pt[0] + w5, pt[1] + h5), (255,0,0), 1)
            elif pt[2] == 'H':
                cv2.rectangle(img, (pt[0],pt[1]), (pt[0] + w0, pt[1] + h0), (255,0,200), 1)
            elif pt[2] == 'B':
                cv2.rectangle(img, (pt[0],pt[1]), (pt[0] + w1, pt[1] + h1), (100,100,200), 1)
    #save image with rectangles in a file
    cv2.imwrite('output.png',img)
    #reverse the list to make it in the order of the image
    ls.reverse()
    #create h for list_to_music function
    h = [h0, h1, h2, h3, h4, h5]
    #use list_to_music function to create music
    list_to_music(ls, h, num, music, lines, height)

# Player
<div dir='rtl'>
در این بخش با استفاده از توابع داده شده در فایل پروژه و با استفاده از رابط های گرافیکی پایتون می خواهیم پخش کننده موسیقی را ایجاد کنیم.
</div>

### Functions For Play
<div dir='rtl'>
    در این بخش تابع های داده شده در فایل پروژه آورده شده اند.
</div>

In [6]:
notes_base = 2**(np.arange(12)/12)*27.5
notes_duration = np.array([3200, 1600, 800, 400, 200, 100])*0.7
notes_ann = ['A', 'A#', 'B', 'C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'G#']

def sin_wave(f, n, fs):
    x = np.linspace(0, 2*np.pi, n)
    ring = 30 
    xp = np.linspace(0, -1*(n*ring/fs), n)
    y = np.sin(x*f*(n/fs))*np.exp(xp)
    z = np.zeros([n, 2])
    z[:, 0] = y
    z[:, 1] = y
    return z

def play_note(note_id, octave, dur, fs):
    if (note_id < 3) :
        octave += 1
    y = sin_wave(notes_base[note_id]*2**octave, int(notes_duration[dur]*(fs/1000)), fs)
    sd.play(y, fs)
    sd.wait()
    return 

def put_note(note_id, octave, dur, fs):
    if (note_id < 3) :
        octave += 1
    y = sin_wave(notes_base[note_id]*2**octave, int(notes_duration[dur]*(fs/1000)), fs)
    return y

def get_music(music_notes, fs):
    m = []
    for item in music_notes:
        y = put_note(item[0], item[1], item[2], fs)
        m.append(y)
    m = np.concatenate(m, 0)
    return m

### Create MP3 File
<div dir='rtl'>
    در این بخش با استفاده از تابع final_func نوشته شده در بخش image processing و تابع get_music در بخش Functions For Play لیست music را تشکیل داده و آن را به یک numpy_array تبدیل کرده ایم و سپس با استفاده از توابع کتابخانه pydub این numpy_array را به یک فایل mp3 تبدیل می کنیم.
</div>

In [7]:
def convert_to_mp3(fs, address, normalized=True):
    file = cv2.imread(address)
    lines, height, num, last_length, length = line_detector(file) #find the horizontal lines
    ls = []
    for i in range(num):
        ls.append([]) 
    music = []
    final_func(file, lines, height, num, last_length, length, ls, music) #create music list
    x = get_music(music, fs) #create numpy array
    
    channels = 2 if (x.ndim == 2 and x.shape[1] == 2) else 1
    if normalized:  # normalized array - each item should be a float in [-1, 1)
        y = np.int16(x * 2 ** 15)
    else:
        y = np.int16(x)
    song = pydub.AudioSegment(y.tobytes(), frame_rate=fs, sample_width=2, channels=channels)
    song.export(address[:-4]+'.mp3', format="mp3", bitrate="320k") #convert to mp3

### Final Part
<div dir='rtl'>
    در این بخش یک رابط گرافیکی برای اجرای موسیقی ایجاد می کنیم. برای اینکار از کتابخانه های tkinter و pygame کمک میگیریم.
    <br>
    نحوه استفاده از این رابط گرافیکی به این صورت است که در کادری که در بالای کلید Convert To MP3 قرار دارد اسم تصویر موردنظر با پسوند (png, jpg, ...) را قرار می دهید و روی کلید Convert To MP3 کلیک می کنید تا فایل MP3 مربوط به تصویر تشکیل شود. پس از آن را زدن کلید Play موسیقی برای شما اجرا می شود و میتوانید پخش آن را با کلید های Pause و Stop و Resume کنترل کنید. برای خروج حتما از کلید Quit استفاده کنید تا فایل های مورد استفاده به طور کامل بسته شوند.
</div>

In [10]:
root = Tk()
root.title('Music Sheet Player')
 
root.geometry("1400x800")
 
pygame.mixer.init()# initialise the pygame

title=Label(root,text="Music Sheet To Sound Converter",bd=9,relief=GROOVE,
            font=("Calibri",50,"bold"),bg="white",fg="black")
title.pack(side=TOP,fill=X)

frame1 = Frame(root, bg='black', width=150)
frame1.pack(pady=20, fill=X)

label1 = Label(frame1, text="Please enter your image file's address", fg='white', bg='black')
label1.pack(pady=5, side=TOP)

input_image = Entry(master=frame1, fg="black", bg="white", width=50)
input_image.pack(pady=5)

def convert_command():
    convert_to_mp3(44100, input_image.get())

def play():
    pygame.mixer.music.load(input_image.get()[:-4]+'.mp3')
    pygame.mixer.music.play(loops=0)

def pause():
    pygame.mixer.music.pause()
    
def resume():
    pygame.mixer.music.unpause()

def stop():
    pygame.mixer.music.stop()

def quit():
    pygame.mixer.music.unload()
    root.destroy()

convert_button = Button(frame1, text="Convert To MP3", font=("Helvetica", 20), command=convert_command, bg='#DCDCDC')
convert_button.pack(pady=5, side=BOTTOM, fill='none', expand=True)
    
frame2 = Frame(root, bg='#A9A9A9', width=150)
frame2.pack(pady=20, fill=X)

play_button = Button(frame2, text="Play", font=("Calibri", 32), command=play, bg='#DCDCDC', width=20)
play_button.pack(pady=20, fill='none', expand=True)

frame3 = Frame(frame2, bg="#A9A9A9")
frame3.pack(pady=20, fill=X)

pause_button = Button(frame3, text="Pause", font=("Calibri", 20), command=pause, bg='#DCDCDC', width=20)
pause_button.pack(fill=Y, side=LEFT)

resume_button = Button(frame3, text="Resume", font=("Calibri", 20), command=resume, bg='#DCDCDC', width=20)
resume_button.pack(fill=Y, side=RIGHT)

stop_button = Button(frame3, text="Stop", font=("Calibri", 20), command=stop, bg='#DCDCDC', width=20)
stop_button.pack(fill=Y, expand=True)

quit_button = Button(root, text="Quit", font=("Calibri", 20), command=quit, bg='black', fg='white')
quit_button.pack(pady=30)

root.mainloop()